**SIN-393 - Introdução à Visão Computacional (2022-2)**

# Projeto Classificação de Imagens

Nome: Caio da Silva de Miranda

Matrícula: 6368

---

## Importando as bibliotecas 
---

In [ ]:
import os

import numpy as np
from skimage import util, transform, filters, color, measure, morphology
from sklearn import model_selection, neighbors, metrics, preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
np.random.seed(393)

from skimage import io

%matplotlib notebook

## Preparando o conjunto de dados do projeto
---
* Vamos utilizar um conjunto de dados fornecido pelo professor contendo:
* 4 Classes:
    * 0 apple;
    * 1 bat;
    * 2 beetle;
    * 3 bone;
* 720 objetos:   
    * 180 objetos de cada classe.

## Importando Datasets necessários

In [ ]:
# Cada conjunto possui 4 classes (apple, bat, beetle, bone)
datasetTeste = 'datasets/Test'

datasetTreino = 'datasets/Train'

## Checagem

In [ ]:
# Checagem para saber se os datasets foram recebidos corretamente
print(datasetTeste)

print(datasetTreino)

## Carregando as imagens de ambos Datasets

### Dataset Treino

In [ ]:
# Lista das pastas na pasta 'Train' (classes)
classes_list = os.listdir(datasetTreino)

# Lista com as imagens no dataset
image_list = []
# Lista com os rótulos das imagens
label_list = []

# Lista com os nomes das imagens
filename_list_ = []

# Percorre as classes do dataset
for classe in classes_list:
    
    # Listagem de todas as imagens na pasta daquela classe
    filename_list = os.listdir(os.path.join(datasetTreino, classe))
    
    # Percorre os arquivos na pasta atual
    for filename in filename_list:
        # Carrega a imagem
        img_temp = io.imread(os.path.join(datasetTreino, classe, filename))
        
        # Adiciona a imagem a lista de imagens
        image_list.append(img_temp)
        
        # Adiciona o rótulo da imagem à lista de rótulos
        label_list.append(classe)
        
        # Adiciona o nome da imagem à uma lista (para fins de visualização)
        filename_list_.append(filename)

In [ ]:
# Lista com os rótulos (classes) das imagens        
print(label_list)

### Dataset Teste

In [ ]:
# Lista das pastas na pasta 'Test' (classes)
classes_list2 = os.listdir(datasetTeste)

# Lista com as imagens no dataset
image_list2 = []
# Lista com os rótulos das imagens
label_list2 = []

# Lista com os nomes das imagens
filename_list_2 = []

# Percorre as classes do dataset
for classe in classes_list2:
    
    # Listagem de todas as imagens na pasta daquela classe
    filename_list2 = os.listdir(os.path.join(datasetTeste, classe))
    
    # Percorre os arquivos na pasta atual
    for filename in filename_list2:
        # Carrega a imagem
        img_temp = io.imread(os.path.join(datasetTeste, classe, filename))
        
        # Adiciona a imagem a lista de imagens
        image_list2.append(img_temp)
        
        # Adiciona o rótulo da imagem à lista de rótulos
        label_list2.append(classe)
        
        # Adiciona o nome da imagem à uma lista (para fins de visualização)
        filename_list_2.append(filename)

In [ ]:
# Lista com os rótulos das imagens        
print(label_list2)

### Convertendo os nomes das classes para índices numéricos

### Treino

In [ ]:
# Indices das classes dos objetos do dataset
_, _, label_list_idx = np.unique(label_list, return_index=True, return_inverse=True)

print(type(label_list_idx))
print(label_list_idx)

### Teste

In [ ]:
# Indices das classes dos objetos do dataset
_, _, label_list_idx2 = np.unique(label_list2, return_index=True, return_inverse=True)

print(type(label_list_idx2))
print(label_list_idx2)

## Extraindo algumas caracteristicas das imagens
----
* Vamos extrair algumas caracteristicas de forma dos objetos nas imagens.
    * Área, maior eixo, menor eixo e solidez.
* Primeiramente precisamos criar um vetor que receberá as características.
* Também será necessário criar um contador para sabermos quantas imagens tiveram problemas durante o processo de extração, e mais à frente calcularmos a diferença que cada classe teve, ou seja, identificar o numero total de imagens bem sucedidas e que servirão para a procedência do projeto.

### Extraindo do Dataset Treino

In [ ]:
# Nomes das caracteristicas computadas
features = ['area', 'major_axis', 'minor_axis', 'solidity']

In [ ]:
# Arranjo 2D com as caracteristicas das imagens
#   Cada linha armazena informações sobre uma imagem. Cada coluna armazena uma caracteristica.
#   [ [area, major_axis, minor_axis, solidity] ]
feature_mat = []

# Lista com as imagens segmentadas (binárias)
seg_list = []

# Lista com os rótulos das imagens
list_label = []

#Contador de imagens que obtiveram problemas durante extração de características
errorcount = []

for i, (image, label) in enumerate(zip(image_list, label_list)):
    # DEBUG
    print('Imagem {} - classe {}'.format(i, label))
    
    # Adiciona o rótulos (label) da imagem à lista
    list_label.append(label)
    
    # Calcula uma lista de propriedades (características) dos objetos na imagem
    props = measure.regionprops(image.astype(int))
    
    ###print(len(props))
    if len(props) != 1:
        
        errorcount.append((i,label))
        print(f'ERRO de segmentação: {len(props)}')
        continue

    # Itera pelas propriedades computadas
    for prop in props:
        # Prop. 0: Area
        area = prop.area
       
        # Prop. 1: Maior eixo
        major_axis = prop.major_axis_length
        
        # Prop. 2: Menor eixo
        minor_axis = prop.minor_axis_length 
        
        # Prop. 3: Solidez
        solidity = prop.solidity 

        # Monta o vetor de caracteristicas deste objeto.
        feature_list = [area, major_axis, minor_axis, solidity]
    
    # Adiciona as caracteristicas desta imagem na matriz de caracteristicas
    feature_mat.append(feature_list)

### Exibindo imagens mal sucedidas

In [ ]:
print(errorcount)

In [ ]:
# Converte a lista de caracteristicas para um arranjo NumPy
feature_map = np.array(feature_mat)

# Imprime a matriz de caracteristica
with np.printoptions(precision=4, suppress=True):
    print(feature_map)

In [ ]:
# Algumas estatisticas sobre o conjunto de caracteristicas
with np.printoptions(precision=4, suppress=True):
    print(feature_map.min(0))
    print(feature_map.max(0))
    print(feature_map.mean(0))
    print(feature_map.std(0))

### Extraindo do Dataset Teste

In [ ]:
# Nomes das caracteristicas computadas
features2 = ['area2', 'major_axis2', 'minor_axis2', 'solidity2']

In [ ]:
# Arranjo 2D com as caracteristicas das imagens
#   Cada linha armazena informações sobre uma imagem. Cada coluna armazena uma caracteristica.
#   [ [area, major_axis, minor_axis, solidity] ]
feature_mat2 = []

# Lista com as imagens segmentadas (binárias)
seg_list2 = []

# Lista com os rótulos das imagens
list_label2 = []

#Contador de erros
errorcount2 = []

for i2, (image2, label2) in enumerate(zip(image_list2, label_list2)):
    # DEBUG
    print('Imagem {} - classe {}'.format(i2, label2))
    
    # Adiciona o rótulos (label) da imagem à lista
    list_label2.append(label2)
    
    # Calcula uma lista de propriedades (características) dos objetos na imagem
    props2 = measure.regionprops(image2.astype(int))
    
    ###print(len(props))
    if len(props2) != 1:
        
        errorcount2.append((i2,label2))
        print(f'ERRO de segmentação: {len(props2)}')
        continue

    # Itera pelas propriedades computadas
    for prop2 in props2:
        # Prop. 0: Area
        area2 = prop2.area
       
        # Prop. 1: Maior eixo
        major_axis2 = prop2.major_axis_length
        
        # Prop. 2: Menor eixo
        minor_axis2 = prop2.minor_axis_length 
        
        # Prop. 3: Solidez
        solidity2 = prop2.solidity 

        # Monta o vetor de caracteristicas deste objeto.
        feature_list2 = [area2, major_axis2, minor_axis2, solidity2]
    
    # Adiciona as caracteristicas desta imagem na matriz de caracteristicas
    feature_mat2.append(feature_list2)

### Exibindo imagens mal sucedidas

In [ ]:
print(errorcount2)

In [ ]:
# Converte a lista de caracteristicas para um arranjo NumPy
feature_map2 = np.array(feature_mat2)

# Imprime a matriz de caracteristica
with np.printoptions(precision=4, suppress=True):
    print(feature_map2)

In [ ]:
# Algumas estatisticas sobre o conjunto de caracteristicas
with np.printoptions(precision=4, suppress=True):
    print(feature_map2.min(0))
    print(feature_map2.max(0))
    print(feature_map2.mean(0))
    print(feature_map2.std(0))

### Preparando os dados para serem computados
---

* Aqui estamos calculando a diferença para cada classe do dataset completo, pelas imagens que não passaram do processo de extração, caso alguma tenha problema durante o processo de segmentação. Foi notado que uma grande quantidade de imagens tinham tido problemas durante o processo de extração devido ao tipo de imagem (*float != int*) e foi corrigido na  função measure.regioprops com um método simples.

### Treino

In [ ]:
cont_apple = 0
cont_beetle = 0
cont_bat = 0
cont_bone = 0

for num, classe in errorcount:
    if classe == 'apple':
        cont_apple = cont_apple + 1
    if classe == 'beetle':
        cont_beetle = cont_beetle + 1
    if classe == 'bat':
        cont_bat = cont_bat + 1
    if classe == 'bone':
        cont_bone = cont_bone + 1

In [ ]:
list_labels_treino = list()
for i in range(126 - cont_apple):
    list_labels_treino.append('apple')
for i in range(126 - cont_beetle):
    list_labels_treino.append('beetle')
for i in range(126 - cont_bat):
    list_labels_treino.append('bat')
for i in range(126 - cont_bone):
    list_labels_treino.append('bone')

In [ ]:
print(cont_apple)
print(cont_beetle)
print(cont_bat)
print(cont_bone)

### Teste

In [ ]:
cont_apple = 0
cont_beetle = 0
cont_bat = 0
cont_bone = 0

for num, classe in errorcount2:
    if classe == 'apple':
        cont_apple = cont_apple + 1
    if classe == 'beetle':
        cont_beetle = cont_beetle + 1
    if classe == 'bat':
        cont_bat = cont_bat + 1
    if classe == 'bone':
        cont_bone = cont_bone + 1

In [ ]:
list_labels_teste = list()
for i in range(54 - cont_apple):
    list_labels_teste.append('apple')
for i in range(54 - cont_beetle):
    list_labels_teste.append('beetle')
for i in range(54 - cont_bat):
    list_labels_teste.append('bat')
for i in range(54 - cont_bone):
    list_labels_teste.append('bone')

In [ ]:
print(cont_apple)
print(cont_beetle)
print(cont_bat)
print(cont_bone)

### Plotando as caracteristicas computadas

### Treino

In [ ]:
df = pd.DataFrame(feature_map, columns=features)

df['class'] = list_labels_treino

### print(df)
display(df)

In [ ]:
g = sns.PairGrid(df, hue='class', vars=features)
g.fig.set_size_inches(8, 8)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend()

### Teste

In [ ]:
df = pd.DataFrame(feature_map2, columns=features2)

df['class'] = list_labels_teste

### print(df)
display(df)

In [ ]:
g = sns.PairGrid(df, hue='class', vars=features2)
g.fig.set_size_inches(8, 8)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend()

## Normalizando as caracteristicas
---

### Normalizando para dataset Treino

In [ ]:
with np.printoptions(precision=4, suppress=True):
    # Média das caracteristicas do conjunto de imagens
    print('Média:')
    print(feature_map.mean(0))
    # Desvio padrão das caracteroisticas do conjunto de imagens
    print('Desvio padrão:')
    print(feature_map.std(0))

In [ ]:
# Transformada Normal de Caracteristicas
feature_map_norm = (feature_map - feature_map.mean(0)) / feature_map.std(0)

print(feature_map_norm)

In [ ]:
with np.printoptions(precision=4, suppress=True):
    # Média das caracteristicas do conjunto de imagens
    print('Média:')
    print(feature_map_norm.mean(0))
    # Desvio padrão das caracteroisticas do conjunto de imagens
    print('Desvio padrão:')
    print(feature_map_norm.std(0))

### Normalizando para dataset Teste

In [ ]:
with np.printoptions(precision=4, suppress=True):
    # Média das caracteristicas do conjunto de imagens
    print('Média:')
    print(feature_map2.mean(0))
    # Desvio padrão das caracteroisticas do conjunto de imagens
    print('Desvio padrão:')
    print(feature_map2.std(0))

In [ ]:
# Transformada Normal de Caracteristicas
feature_map2_norm = (feature_map2 - feature_map2.mean(0)) / feature_map2.std(0)

print(feature_map2_norm)

In [ ]:
with np.printoptions(precision=4, suppress=True):
    # Média das caracteristicas do conjunto de imagens
    print('Média:')
    print(feature_map2_norm.mean(0))
    # Desvio padrão das caracteroisticas do conjunto de imagens
    print('Desvio padrão:')
    print(feature_map2_norm.std(0))

### Plotando as caracteristicas normalizadas - Treino

In [ ]:
df_norm = pd.DataFrame(feature_map_norm, columns=features)

df_norm['class'] = list_labels_treino
print(df_norm)

In [ ]:
g = sns.PairGrid(df_norm, hue='class', vars=features)
g.fig.set_size_inches(8, 8)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend()

### Plotando as caracteristicas normalizadas - Teste

In [ ]:
df_norm2 = pd.DataFrame(feature_map2_norm, columns=features2)

df_norm2['class'] = list_labels_teste
print(df_norm)

In [ ]:
g = sns.PairGrid(df_norm2, hue='class', vars=features2)
g.fig.set_size_inches(8, 8)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend()

## Validação cruzada - Hold-out
---

* Separa o conjunto de dados em subconjuntos para treinamento, validação e testes. 
    * Neste exemplo, por motivos de simplificação, vamos dividir em treino e testes apenas.

In [ ]:
# Utilizando apenas duas das 4 características: Área e maior-eixo
feature_map_ok = feature_map[:,0:2]

In [ ]:
X_train = feature_map
X_test = feature_map2
y_train = list_labels_treino
y_test = list_labels_teste

### Normalizando as caracteristicas

* A normalização **não** deve ser realizada sobre todo o conjunto de dados. 
    * A normalização deve ser realizada **após** a divisão do conjunto para a validação cruzada.
    * O conjunto de testes não deve ser acessado, nem direta nem indiretamente, durante o treinamento ou durante o ajuste de hiperparâmetros. 
    * A normalização do conjunto de treinamento e também do conjunto de testes deve ser realizado usando apenas a média e o desvio padrão do conjunto de treinamento.

In [ ]:
# Média das caracteristicas do conjunto de treinamento
X_train_mean = X_train.mean(0)
# Desvio padrão das caracteristicas do conjunto de treinamento
X_train_std = X_train.std(0)

with np.printoptions(precision=4, suppress=True):
    print(X_train.mean(0))
    print(X_train.std(0))

In [ ]:
# Transformada Normal de Caracteristicas
X_train_norm = (X_train - X_train_mean) / X_train_std
X_test_norm = (X_test - X_train_mean) / X_train_std

with np.printoptions(precision=4, suppress=True):
    print(X_train_norm)
    print(X_test_norm)

## Classificando usando K-vizinhos mais próximos
---

In [ ]:
# Constrói um classificador do tipo K-NN onde K = 3.
clf = neighbors.KNeighborsClassifier(n_neighbors=3)

# Treinando o classificador
clf.fit(X_train_norm, y_train)

# Testando o classificador
pred = clf.predict(X_test_norm)

## Avaliação de modelo
---

In [ ]:
# Predições e acertos utilizando o classificador
acertos = y_test == pred

print('\n Predição:')
print(pred)
print('\nReal:')
print(y_test)
print('\nAcerto/Erro:')
print(acertos.astype(int))

### Matriz de confusão e o relatório de treinamento

In [ ]:
print('\nMatriz de confusão:')
print(metrics.confusion_matrix(y_test, pred))

print('\nRelatório de classificação:')
print(metrics.classification_report(y_test, pred))

## Otimizando hiperparametros com o conjunto de validação
---
* Não se deve realizar a otimização de hiperparâmetros usando o conjunto de testes.
* Dessa forma, separamos uma parte do conjunto de treinamento para validação.
* Para fazer isso usando o Scikit-learn:
    * Primeiro separamos o conjunte do dados total em teste e treino. 
    * Depois separamos o conjunto de treino em validação e conjunto de testes final. 

* *Exemplo:* 20% para testes, 20% para validação e 60% para treinamento.

```
 +-- Conjunto de dados - 100%
     +-- Conjunto de testes - 20%
     +-- Conjunto de treino 1 - 80%
         +-- Conjunto de validação - 20% do conjunto de dados = 25% do conjunto de treino 1 (0,2 / 0,8 = 0,25)
         +-- Conjunto de treino 2 - 60 % do conjunto de dados = 75% do conjunto de treino 1 (0,8 * 0,75 = 0,6)```

* Entretanto, o exemplo anterior estava muito fácil de resolver, conseguimos acertar 100% dos casos na primeira tentativa.
* Vamos escolher outras caracteristicas, para tornar a tarefa um pouco mais dificil para o nosso classificador.

In [ ]:
# Selecionamos apenas duas caracteristicas: Área e solidez
feature_map_ok = feature_map[:,[0,3]]

* Vamos melhorar a divisão do conjutno de dados também;
    * Vamsos adotar uma divisão estratificada.
        * A divisão estratificada preserva a proporção entre amostras de cada classe nos conjuntos de treino, validação e testes.

In [ ]:
X_train = feature_map_ok
X_test = feature_map2[:,[0,3]]
y_train = list_labels_treino
y_test = list_labels_teste

In [ ]:
# Separa 25% do conjuto de treinamento 1 para validação.
#   -> Equivale a 20% do conjunto completo. 0,2 / 0,8 = 0,25
X_train_2, X_val, y_train_2, y_val = model_selection.train_test_split(X_train, 
                                                                      y_train, 
                                                                      test_size=0.25, 
                                                                      stratify=y_train,
                                                                      random_state=393)

### Normalizando as características
* Obtemos uma estimativa da média e do desvio padrão dos dados a partir do conjunto de treino.

In [ ]:
# Média das caracteristicas do conjunto de treinamento
X_train_2_mean = X_train_2.mean(0)

# Desvio padrão das caracteristicas do conjunto de treinamento
X_train_2_std = X_train_2.std(0)

with np.printoptions(precision=4, suppress=True):
    print(X_train_2.mean(0))
    print(X_train_2.std(0))

* Aqui utilizamos a função disponível no Scikit-learn para fazermos a normalização das características

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train_2)
with np.printoptions(precision=4, suppress=True):
    print(f'Média:  \t {np.array(scaler.mean_)}')
    print(f'Desv. pad.: \t {np.array(scaler.scale_)}')

In [ ]:
X_train_2_norm = scaler.transform(X_train_2)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

with np.printoptions(precision=4, suppress=True):
    print(f'Treino: \t {X_train_norm.mean():.4f} ± {X_train_norm.std():.4f}')
    print(f'Validação: \t {X_val_norm.mean():.4f} ± {X_val_norm.std():.4f}')
    print(f'Teste:   \t {X_test_norm.mean():.4f} ± {X_test_norm.std():.4f}')

### Otimizando o valor de *k*
* Vamos encontrar o melhor valor de k para o K-means em termos de acurácia.
* Vamos testar os seguintes valores de 'k': 1, 3, 5, 7 e 9.

In [ ]:
k_list = [1, 3, 5, 7, 9]

In [ ]:
# Lista com as acurácias de traino
acc_train_list = []
# Lista com as acurácias de validação
acc_val_list = []

for k_ in k_list:
    # Constrói um classificador K-NN. K = k_
    clf = neighbors.KNeighborsClassifier(n_neighbors=k_)

    # Treinando o classificador
    clf.fit(X_train_2_norm, y_train_2)

    # Testando o classificador (usando o conjunto de validação)
    pred = clf.predict(X_val_norm)
    acc_val = metrics.accuracy_score(y_val, pred)
    
    acc_val_list.append(acc_val)
    
    # Testando o classificador (usando o conjunto de treino)
    # **** Apenas para comparar com o resultado da validação ****
    pred_train = clf.predict(X_train_2_norm)
    acc_train = metrics.accuracy_score(y_train_2, pred_train)
    
    acc_train_list.append(acc_train)  

In [ ]:
plt.figure(figsize=(9, 6))

plt.plot(k_list, acc_train_list, 'o', color='blue', label='treino')
plt.plot(k_list, acc_val_list, 'x', color='red', label='validação')
plt.xlabel("Valor de 'k'")
plt.ylabel("Acurácia")
plt.legend(loc='best')

plt.show()

In [ ]:
print('k \t acc. treino \t acc. val')
print('----------------------------')
for k_, acc_t, acc_v in zip(k_list, acc_train_list, acc_val_list):
    print(f'{k_} \t {acc_t:.4f} \t {acc_v:.4f}')

k_best = k_list[np.argmax(acc_val_list)]
print(f'\nMelhor \'k\': {k_best} ({np.max(acc_val_list):.4f} acc.)')

### Utilizando melhor *k* encontrado sob conjunto de Testes

In [ ]:
# Constrói um classificador K-NN. K = k_best
clf = neighbors.KNeighborsClassifier(n_neighbors=k_best)

# Treinando o classificador
clf.fit(X_train_2_norm, y_train_2)

# Testando o classificador (usando o conjunto de TESTES)
pred = clf.predict(X_test_norm)
acc_val = metrics.accuracy_score(y_test, pred)

* Matriz de confusão e relatório de classificação

In [ ]:
print('\nMatriz de confusão:')
print(metrics.confusion_matrix(y_test, pred))

print('\nRelatório de classificação:')
print(metrics.classification_report(y_test, pred))